# Финальный проект: Прогнозирование оттока клиентов

## Неделя 4: Построение и оптимизация модели

** 2017/01/04**

*Юрий Исаков*

В рамках проекта мы будем решать задачу прогнозирования оттока пользователей, или churn prediction. Эта задача заключается в прогнозировании вероятности того, что клиент перестанет пользоваться сервисом в течение некоторого заданного заранее промежутка времени, например, решит перейти к конкуренту или откажется от использования услуги данного типа вообще.

Данные были предоставлены французской телекоммуникационной компанией Orange. В задаче речь идет о клиентских данных, поэтому данные были предварительно обфусцированы и анонимизированны: из датасета убрана любая персональная информация, позволяющая идентифицировать пользователей, а также не представлены названия и описания переменных, предназначенных для построения прогнозов. Мы будем работать с набором данных orange small dataset. Он состоит из 50 тыс. объектов и включает 230 переменных, из которых первые 190 переменных - числовые, и оставшиеся 40 переменные - категориальные.

---

В этом задании вам предстоит поучаствовать в соревновании на kaggle inclass

Перейдите по ссылке на страницу соревнования: https://inclass.kaggle.com/c/telecom-clients-churn-prediction

И приступайте!

В соревновании вы будете работать с той же выборкой, что и ранее, поэтому воспользуйтесь результатами полученными на предыдущих неделях. Для успешного участия в соревновании необходимо преодолеть по качеству beseline решение.

In [1]:
# библиотеки
import pandas as pd
import numpy as np
import itertools
import xgboost as xgb

# 1. Загрузка данных

In [2]:
train = pd.read_csv("data/orange_small_churn_train_data.csv")
train.loc[train['labels'] == -1, 'labels'] = 0
labels = train['labels'].copy()
test = pd.read_csv('data/orange_small_churn_test_data.csv')

# имена категориальных и числовых переменных
features_num = train.columns[1:191].tolist()
features_cat = train.columns[191:231].tolist()

# приводим типы
train[features_cat] = train[features_cat].astype(object)
test[features_cat] = test[features_cat].astype(object)

In [3]:
# среднее по выборке
baseline = labels.mean()
print labels.mean()

0.0744


# 2. Категориальные переменные

In [4]:
# заменим пропущенные значения в категориальных признаках на свое
my_na = 'fakenan'
train[features_cat] = train[features_cat].fillna(my_na)
test[features_cat] = test[features_cat].fillna(my_na)

In [5]:
# добавляем комбинации
cat_2inter = ["Var218", "Var199", "Var212", "Var192", "Var216", "Var205", "Var204", "Var226",\
              "Var197", "Var210", "Var198", "Var229", "Var217", "Var208"]
features_cmb = []

train_cmb2 = pd.DataFrame()
test_cmb2 = pd.DataFrame()

# 
for cmb in itertools.combinations(cat_2inter, 2):
    feat = cmb[0] + cmb[1]
    features_cmb.append(feat)
    train_cmb2[feat] = train[cmb[0]] + '-' + train[cmb[1]]
    test_cmb2[feat] = test[cmb[0]] + '-' + test[cmb[1]]

train = pd.concat([train, train_cmb2], axis = 1)
test = pd.concat([test, test_cmb2], axis = 1)

train[features_cmb].head()

,Var218Var199,Var218Var212,Var218Var192,Var218Var216,Var218Var205,Var218Var204,Var218Var226,Var218Var197,Var218Var210,Var218Var198,...,Var210Var198,Var210Var229,Var210Var217,Var210Var208,Var198Var229,Var198Var217,Var198Var208,Var229Var217,Var229Var208,Var217Var208
0,cJvF-I1sFbv_0IT,cJvF-JBfYVit4g8,cJvF-NESt0G8EIb,cJvF-TDctq2l,cJvF-09_Q,cJvF-k13i,cJvF-fKCe,cJvF-0LaQ,cJvF-uKAI,cJvF-UaKK0yW,...,uKAI-UaKK0yW,uKAI-fakenan,uKAI-KmRo,uKAI-kIsH,UaKK0yW-fakenan,UaKK0yW-KmRo,UaKK0yW-kIsH,fakenan-KmRo,fakenan-kIsH,KmRo-kIsH
1,cJvF-o64y9zI,cJvF-XfqtO3UdzaXh_,cJvF-P1WvyxLp3Z,cJvF-XTbqizz,cJvF-VpdQ,cJvF-FbIm,cJvF-xb3V,cJvF-YFAj,cJvF-uKAI,cJvF-Bnunsla,...,uKAI-Bnunsla,uKAI-mj86,uKAI-qMoY,uKAI-kIsH,Bnunsla-mj86,Bnunsla-qMoY,Bnunsla-kIsH,mj86-qMoY,mj86-kIsH,qMoY-kIsH
2,UYBR-nQUveAzAF7,UYBR-4kVnq_T26xq1p,UYBR-FoxgUHSK8h,UYBR-pMWBUmQ,UYBR-VpdQ,UYBR-mTeA,UYBR-FSa2,UYBR-TyGl,UYBR-uKAI,UYBR-fhk21Ss,...,uKAI-fhk21Ss,uKAI-mj86,uKAI-qLXr,uKAI-kIsH,fhk21Ss-mj86,fhk21Ss-qLXr,fhk21Ss-kIsH,mj86-qLXr,mj86-kIsH,qLXr-kIsH
3,cJvF-LWyxgtXeJL,cJvF-NhsEn4L,cJvF-vNEvyxLp3Z,cJvF-kZJtVhC,cJvF-VpdQ,cJvF-vzJD,cJvF-xb3V,cJvF-0Xwj,cJvF-uKAI,cJvF-uoZk2Zj,...,uKAI-uoZk2Zj,uKAI-fakenan,uKAI-JC0e,uKAI-kIsH,uoZk2Zj-fakenan,uoZk2Zj-JC0e,uoZk2Zj-kIsH,fakenan-JC0e,fakenan-kIsH,JC0e-kIsH
4,cJvF-ZIXKpoNpqq,cJvF-NhsEn4L,cJvF-4e7gUH7IEC,cJvF-NGZXfGp,cJvF-sJzTlal,cJvF-m_h1,cJvF-WqMG,cJvF-vSNn,cJvF-uKAI,cJvF-kugYdIL,...,uKAI-kugYdIL,uKAI-fakenan,uKAI-064o,uKAI-kIsH,kugYdIL-fakenan,kugYdIL-064o,kugYdIL-kIsH,fakenan-064o,fakenan-kIsH,064o-kIsH


In [6]:
# категориальные переменные для модели
feat_cat_use = ['Var191', 'Var192', 'Var193', 'Var194', 'Var195', 'Var197', 'Var198', 'Var199', 'Var200', 
                'Var202', 'Var203', 'Var204', 'Var205','Var206', 'Var207', 'Var208', 'Var210', 'Var211', 'Var212', 
                'Var216', 'Var217', 'Var218', 'Var219', 'Var221', 'Var223', 'Var224', 'Var225', 'Var226', 'Var227', 
                'Var228', 'Var229'] + features_cmb

In [7]:
# тест: замена сгенерированных значений вероятностью положительного класса по значениям
for feat in feat_cat_use:
    tm = train.groupby([feat])['labels'].mean()
    test[feat] = test[feat].map(tm)

# чего не встретилось заменим на общее среднее
test[feat_cat_use] = test[feat_cat_use].fillna(baseline)

In [8]:
%%time
# трейн: замена сгенерированных значений вероятностью положительного класса по значениям
zebra = 4
for feat in feat_cat_use:
    train_feat = np.array([np.nan] * train.shape[0])
    
    # чтобы не переобучиться, разбобьем выборку на несколько частей. тогда средние для первой части будут
    # вычисляться по 2, 3 и 4 и т.д.
    for i in range(zebra):
        idx_rplc = range(train.shape[0])[i::zebra]
        idx_tm = list(set(range(train.shape[0])) - set(idx_rplc)) # ппц, как это правильно сделать на питоне?)
        tm = train.ix[idx_tm, :].groupby([feat])['labels'].mean()
        train_feat[idx_rplc] = train.ix[idx_rplc, feat].map(tm)
    
    train[feat] = train_feat

# чего не встретилось заменим на общее среднее
train[feat_cat_use] = train[feat_cat_use].fillna(baseline)

CPU times: user 55.5 s, sys: 18.1 s, total: 1min 13s
Wall time: 1min 15s


# 3. Числовые переменные

In [9]:
# числовые переменные для модели
feat_num_use = ['Var101', 'Var102', 'Var106', 'Var109', 'Var111', 'Var112', 'Var113', 'Var115', 'Var119', 'Var121', 
                'Var123', 'Var125', 'Var126', 'Var129', 'Var13', 'Var132', 'Var133', 'Var134', 'Var135', 'Var136', 
                'Var140', 'Var144', 'Var146', 'Var149', 'Var152', 'Var153', 'Var158', 'Var16', 'Var160', 'Var163', 
                'Var164', 'Var165', 'Var166', 'Var168', 'Var180', 'Var181', 'Var182', 'Var183', 'Var187', 'Var188', 
                'Var189', 'Var21', 'Var22','Var24', 'Var25', 'Var28', 'Var35', 'Var38', 'Var40', 'Var41', 'Var44', 
                'Var45', 'Var51', 'Var53', 'Var56', 'Var57', 'Var58', 'Var59', 'Var6', 'Var61', 'Var63', 'Var65', 
                'Var69', 'Var7', 'Var72', 'Var73', 'Var74', 'Var76', 'Var78', 'Var81', 'Var82', 'Var83', 'Var85', 
                'Var86', 'Var94', 'Var190']

In [10]:
# заменяем пропущенные значения средними
num_means = train[feat_num_use].mean()
train[feat_num_use] = train[feat_num_use].fillna(num_means)
test[feat_num_use] = test[feat_num_use].fillna(num_means)

# 4. Построение модели

Данные готовы для построения модели, вот как выглядят первые строки тренировочной выборки:

In [11]:
# признаки, используемые в модели
feat_use = feat_num_use + feat_cat_use
train[feat_use].head()

,Var101,Var102,Var106,Var109,Var111,Var112,Var113,Var115,Var119,Var121,...,Var210Var198,Var210Var229,Var210Var217,Var210Var208,Var198Var229,Var198Var217,Var198Var208,Var229Var217,Var229Var208,Var217Var208
0,19.131429,29842.227736,38918.903529,144.0,296073.607797,144.0,-1209960.0,30.6571,1660.0,6.989474,...,0.000000,0.084698,0.000000,0.070227,0.000000,0.0744,0.000000,0.0000,0.086048,0.0000
1,19.131429,29842.227736,38918.903529,80.0,296073.607797,72.0,417932.0,30.6571,1025.0,6.989474,...,0.074400,0.051356,0.074400,0.070847,0.074400,0.0744,0.074400,0.0744,0.054525,0.0744
2,19.131429,29842.227736,38918.903529,40.0,296073.607797,48.0,-124655.2,30.6571,590.0,6.989474,...,0.034357,0.054951,0.111111,0.071567,0.038797,0.0744,0.034166,0.0000,0.055515,0.1250
3,19.131429,29842.227736,38918.903529,32.0,296073.607797,32.0,378473.6,30.6571,1435.0,6.989474,...,0.052632,0.089291,0.074400,0.072729,0.090909,0.0744,0.058824,0.0744,0.089704,0.0744
4,19.131429,29842.227736,38918.903529,32.0,296073.607797,8.0,142602.4,30.6571,490.0,6.989474,...,0.000000,0.084698,0.000000,0.070227,0.000000,0.0744,0.000000,0.0000,0.086048,0.0000


In [12]:
# формат для данных для хгбуста
train_xgb = xgb.DMatrix(train[feat_use].as_matrix(), train['labels'])
test_xgb = xgb.DMatrix(test[feat_use].as_matrix())

In [13]:
# параметры обучения
param = {'eta': 0.03, 'subsample': 0.7, 'colsample_bytree': 1, 
         'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight': 5, 'eval_metric': 'auc'}
evals = [(train_xgb, 'train')]

In [14]:
# обучим модель, будем выводить метрику на обучающей выборке
bst = xgb.train(param, train_xgb, num_boost_round = 351, verbose_eval = 50, evals = evals)

[0]	train-auc:0.675301
[50]	train-auc:0.738759
[100]	train-auc:0.754419
[150]	train-auc:0.765895
[200]	train-auc:0.775567
[250]	train-auc:0.784437
[300]	train-auc:0.792455
[350]	train-auc:0.799271


In [15]:
# посылочка
y_test = pd.DataFrame({'result': bst.predict(test_xgb), 'ID': test.index})
y_test.to_csv('submission.csv', index = False)

# 5. Выводы

Мне не удалось побить собственный бейслайн (3 неделя). Во-первых, там был другой датасет :), а во-вторых, там было приличное переобучение, т.к. я использовал результаты анализа до выделения отложенной выборки ;) Воспроизведя построение бейслайнов, представленное решение ощутимо сильнее (0.65 vs 0.74 по отложенной выборке), но по лидерборду это отличия не такие выраженные (0.70).

Пс: на кросс-валидации значение метрики составляло ~0.745 и оно хорошо согласовывалось со значением на отложенной выборке. На паблике значение ощутимо меньше для самых разных моделей и не всегда оно объясняется не только метрикой на кросс-валидации, но и на отложенной выборке. Возможно тестовая выборка из _немного другого распределения_.

<img src=https://lh3.googleusercontent.com/T3u97Lq-eiszc89DiHofLZsSSHzZ42gWrohqCULbodqXTGDd0AOJXCsV4eRrqOQQKLqhrGZxg53Z3xFZ2bRUKNZOjb0Ipz4lNTz3FHNZ4SdPt79QTHsrZmVLaEgS4MZgLLCImLiKgmeXTjwcHF9YMpkZqT0FXNGYs3HSKFeCml7oWjllVQ1GbEPeO-58XddsQ-swJW-T2CsO0PNsAyok7J_bl9R5b0EgVGXkgWx2rETo2qwIwPnX5_2gW3YEQ06tIAHs8_NGLM3mUSti-MymjxjV7i9XiXfeB-inGz9w4OhKn8FpGfvDmZ5ITUc1P56ZyZgC_GQ83VcTYp3llrdvPON5AA1Fut-EVwWDQ2bcI2ozvUFZhp00blZDVXHss5NU9IJKd-mVy9gLyBhgPVaBdOzfb0tx0T3zEJE--mUNIWnkGcjdV9q3me7dt45NdflwqT2i2LKbc1VeJCp1b5Gc3jdrK3KDxXbve9CZLwTTS29gfmYlGUl7M85Q-PvX3ZCIZP7RlCtHKdy4kyPslTCZ52e78ZAkwyrXMcX9xRo5dOsv4aIbThlEwMrjP2RR1MRyHiXkiR1Af_I6J7bPWDbugkX-eSpRm7ExSWqLBFtJnqsERgdVwtipvnwvIAcT_k1rj201sSFlPP06FURZo0694XLUze54rI04Gbq10JEf3g=w1938-h1454-no>